In [1]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')
model = BertForSequenceClassification.from_pretrained('ProsusAI/finbert')

In [2]:
txt = """
I would like to get your all  thoughts on the bond yield increase this week.  I am not worried about the market downturn but the sudden increase in yields. On 2/16 the 10 year bonds yields increased by almost  9 percent and on 2/19 the yield increased by almost 5 percent.

Key Points from the CNBC Article:

* **The “taper tantrum” in 2013 was a sudden spike in Treasury yields due to market panic after the Federal Reserve announced that it would begin tapering its quantitative easing program.**
* **Major central banks around the world have cut interest rates to historic lows and launched unprecedented quantities of asset purchases in a bid to shore up the economy throughout the pandemic.**
* **However, the recent rise in yields suggests that some investors are starting to anticipate a tightening of policy sooner than anticipated to accommodate a potential rise in inflation.**

The recent rise in bond yields and U.S. inflation expectations has some investors wary that a repeat of the 2013 “taper tantrum” could be on the horizon.

The benchmark U.S. 10-year Treasury note climbed above 1.3% for the first time since February 2020 earlier this week, while the 30-year bond also hit its highest level for a year. Yields move inversely to bond prices.

Yields tend to rise in lockstep with inflation expectations, which have reached their highest levels in a decade in the U.S., powered by increased prospects of a large fiscal stimulus package, progress on vaccine rollouts and pent-up consumer demand.

The “taper tantrum” in 2013 was a sudden spike in Treasury yields due to market panic after the Federal Reserve announced that it would begin tapering its quantitative easing program.

Major central banks around the world have cut interest rates to historic lows and launched unprecedented quantities of asset purchases in a bid to shore up the economy throughout the pandemic. The Fed and others have maintained supportive tones in recent policy meetings, vowing to keep financial conditions loose as the global economy looks to emerge from the Covid-19 pandemic.

However, the recent rise in yields suggests that some investors are starting to anticipate a tightening of policy sooner than anticipated to accommodate a potential rise in inflation.

With central bank support removed, bonds usually fall in price which sends yields higher. This can also spill over into stock markets as higher interest rates means more debt servicing for firms, causing traders to reassess the investing environment.

“The supportive stance from policymakers will likely remain in place until the vaccines have paved a way to some return to normality,” said Shane Balkham, chief investment officer at Beaufort Investment, in a research note this week.

“However, there will be a risk of another ‘taper tantrum’ similar to the one we witnessed in 2013, and this is our main focus for 2021,” Balkham projected, should policymakers begin to unwind this stimulus.

Long-term bond yields in Japan and Europe followed U.S. Treasurys higher toward the end of the week as bondholders shifted their portfolios.

“The fear is that these assets are priced to perfection when the ECB and Fed might eventually taper,” said Sebastien Galy, senior macro strategist at Nordea Asset Management, in a research note entitled “Little taper tantrum.”

“The odds of tapering are helped in the United States by better retail sales after four months of disappointment and the expectation of large issuance from the $1.9 trillion fiscal package.”

Galy suggested the Fed would likely extend the duration on its asset purchases, moderating the upward momentum in inflation.

“Equity markets have reacted negatively to higher yield as it offers an alternative to the dividend yield and a higher discount to long-term cash flows, making them focus more on medium-term growth such as cyclicals” he said. Cyclicals are stocks whose performance tends to align with economic cycles.

Galy expects this process to be more marked in the second half of the year when economic growth picks up, increasing the potential for tapering.

## Tapering in the U.S., but not Europe

Allianz CEO Oliver Bäte told CNBC on Friday that there was a geographical divergence in how the German insurer is thinking about the prospect of interest rate hikes.

“One is Europe, where we continue to have financial repression, where the ECB continues to buy up to the max in order to minimize spreads between the north and the south — the strong balance sheets and the weak ones — and at some point somebody will have to pay the price for that, but in the short term I don’t see any spike in interest rates,” Bäte said, adding that the situation is different stateside.

“Because of the massive programs that have happened, the stimulus that is happening, the dollar being the world’s reserve currency, there is clearly a trend to stoke inflation and it is going to come. Again, I don’t know when and how, but the interest rates have been steepening and they should be steepening further.”

## Rising yields a ‘normal feature’

However, not all analysts are convinced that the rise in bond yields is material for markets. In a note Friday, Barclays Head of European Equity Strategy Emmanuel Cau suggested that rising bond yields were overdue, as they had been lagging the improving macroeconomic outlook for the second half of 2021, and said they were a “normal feature” of economic recovery.

“With the key drivers of inflation pointing up, the prospect of even more fiscal stimulus in the U.S. and pent up demand propelled by high excess savings, it seems right for bond yields to catch-up with other more advanced reflation trades,” Cau said, adding that central banks remain “firmly on hold” given the balance of risks.

He argued that the steepening yield curve is “typical at the early stages of the cycle,” and that so long as vaccine rollouts are successful, growth continues to tick upward and central banks remain cautious, reflationary moves across asset classes look “justified” and equities should be able to withstand higher rates.

“Of course, after the strong move of the last few weeks, equities could mark a pause as many sectors that have rallied with yields look overbought, like commodities and banks,” Cau said.

“But at this stage, we think rising yields are more a confirmation of the equity bull market than a threat, so dips should continue to be bought.”
"""

In [3]:
tokens = tokenizer.encode_plus(txt, add_special_tokens = False, return_tensors = 'pt')

print(len(tokens['input_ids']))
tokens

Token indices sequence length is longer than the specified maximum sequence length for this model (1345 > 512). Running this sequence through the model will result in indexing errors


1


{'input_ids': tensor([[1045, 2052, 2066,  ..., 4149, 1012, 1524]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}

In [4]:
a = torch.arange(10)
a

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [5]:
torch.split(a, 4)

(tensor([0, 1, 2, 3]), tensor([4, 5, 6, 7]), tensor([8, 9]))

In [7]:
input_ids_chunks = tokens['input_ids'][0].split(510)
attention_mask_chunks = tokens['attention_mask'][0].split(510)

In [13]:
len(attention_mask_chunks[0])

510

In [15]:
chunksize = 512

input_ids_chunks = list(tokens['input_ids'][0].split(chunksize - 2))
attention_mask_chunks = list(tokens['attention_mask'][0].split(chunksize - 2))

for i in range(len(input_ids_chunks)):
    input_ids_chunks[i] = torch.cat(
        [torch.Tensor([101]), input_ids_chunks[i], torch.Tensor([102])]
    )
    attention_mask_chunks[i] = torch.cat(
        [torch.Tensor([1]), attention_mask_chunks[i], torch.Tensor([1])]
    )
    
    pad_len = chunksize - input_ids_chunks[i].shape[0]
    
    if pad_len > 0:
        input_ids_chunks[i] = torch.cat(
            [input_ids_chunks[i], torch.Tensor([0] * pad_len)]
        )
        attention_mask_chunks[i] = torch.cat(
            [attention_mask_chunks[i], torch.Tensor([0] * pad_len)]
        )

In [18]:
input_ids = torch.stack(input_ids_chunks)
attention_mask = torch.stack(attention_mask_chunks)

In [19]:
input_ids

tensor([[  101.,  1045.,  2052.,  ...,  1012.,  1523.,   102.],
        [  101.,  1996., 16408.,  ...,  2272.,  1012.,   102.],
        [  101.,  2153.,  1010.,  ...,     0.,     0.,     0.]])

In [23]:
input_dict = {
    'input_ids': input_ids.long(),
    'attention_mask': attention_mask.int()
}

In [24]:
input_dict

{'input_ids': tensor([[  101,  1045,  2052,  ...,  1012,  1523,   102],
         [  101,  1996, 16408,  ...,  2272,  1012,   102],
         [  101,  2153,  1010,  ...,     0,     0,     0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0]], dtype=torch.int32)}

In [25]:
outputs = model(**input_dict)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.2807,  1.4917, -1.3592],
        [ 0.0747,  0.2922, -0.7954],
        [ 1.0205, -0.2699, -1.3167]], grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)

In [26]:
probs = torch.nn.functional.softmax(outputs[0], dim=-1)
probs

tensor([[0.1384, 0.8145, 0.0471],
        [0.3757, 0.4670, 0.1574],
        [0.7290, 0.2006, 0.0704]], grad_fn=<SoftmaxBackward>)

In [27]:
probs = probs.mean(dim=0)
probs

tensor([0.4144, 0.4940, 0.0916], grad_fn=<MeanBackward1>)